In [2]:
#Installing all necessary libaries
!pip install langchain
!pip install transformers
!pip install huggingface_hub
!pip install "unstructured[all-docs]"
!pip install faiss-gpu
!pip install faiss-cpu
!pip install sentence-transformers
!pip install accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 97.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 66.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=49d2f63b9ce4985ff9811677b6def0176bf8f2bdbbb7876bb969aeda3f672513
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.9 MB/s eta 0:00:00


IMPORTS

In [1]:

# Libraries for document processing, embedding, and question-answering
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from huggingface_hub import notebook_login
from transformers import pipeline
from transformers import AutoConfig, AutoModelForCausalLM,AutoTokenizer
from langchain import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter

# General purpose libraries and utilities
import textwrap
import torch
import sys
import os

### SETUP

In [2]:
# Setting up HuggingFace Hub authentication
os.environ['HuggingFaceHub_API_Token']= 'enter_api_token_here'
notebook_login()

DOCUMENT LOADING

In [3]:
# Load documents from a given file (e.g., 'file.pdf')
loader = UnstructuredFileLoader('file.pdf')
documents = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


DOCUMENT PROCESSING

In [4]:
# Split documents into manageable chunks for better processing.
# This helps in extracting and working with specific passages from the text.
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=50)
text_chunks=text_splitter.split_documents(documents)

EMBEDDING

In [5]:
# Generating embeddings using a pre-trained model from HuggingFace.
# The embeddings will represent the semantic content of our text chunks.
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',model_kwargs={'device': 'cuda'})

VECTOR STORE

In [6]:
# Storing the embeddings in a FAISS vector store.
# FAISS allows efficient similarity search and retrieval of the chunks.
vectorstore=FAISS.from_documents(text_chunks, embeddings)

MODEL SETUP FOR QUESTION ANSWERING

In [7]:
# Setting up the tokenizer and model for the LLM
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [8]:
# Defining the pipeline for text generation using the model and tokenizer.
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 1024,
                do_sample=True,
                top_k=10,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [9]:
# Integrating the HuggingFace pipeline with langchain, setting temperature to 0 for deterministic outputs.
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [10]:
# Establishing the retrieval-based QA chain using our previously created vector store for document retrieval.
chain =  RetrievalQA.from_chain_type(llm=llm, chain_type = "stuff",return_source_documents=True, retriever=vectorstore.as_retriever())

In [11]:
# query = "Which teams played in the NFL Kickoff Game to begin the 2022 season, and what was the result?"
# result=chain({"query": query}, return_only_outputs=True)
# wrapped_text = textwrap.fill(result['result'], width=500)
# wrapped_text

QUESTION ANSWERING

In [12]:
# Test
# Looping over questions, retrieving answers and printing them.
questions = [
    "Which teams played in the NFL Kickoff Game to begin the 2022 season, and what was the result?",
    "Why was the game between Buffalo and Cincinnati cancelled?",
    "What new name was adopted by the former Washington Redskins for the 2022 NFL season?",
    "State the career trajectory of Tra Blake leading up to his promotion to replace Tony Corrente as a referee in the 2022 NFL season.",
    "Who was hired as the league's first Asian-American NFL official and from which conference was this individual recruited?",
     "In regard to the inclusive hiring training announced at the NFL Fall League Meeting on October 18, who is mandated to participate and under what circumstances must this training be undertaken?"
]

for query in questions:
    result=chain({"query": query}, return_only_outputs=True)
    wrapped_text = textwrap.fill(result['result'], width=500)
    print(wrapped_text)
    print("*"*50)

 The Los Angeles Rams played the Buffalo Bills in the 2022 NFL Kickoff Game on September 8, 2022, and the Bills won with a score of 31-10.
**************************************************
 The game between Buffalo and Cincinnati was cancelled after Buffalo safety Damar Hamlin suffered a nonfatal cardiac arrest on the field of play. I don't know the answer to the question.
**************************************************
 The former Washington Redskins were renamed the Washington Commanders prior to the start of the 2022 NFL season.
**************************************************
 Tra Blake's career trajectory leading up to his promotion to replace Tony Corrente as a referee in the 2022 NFL season is as follows: * In 2019, Blake worked as an umpire in the Alliance of American Football (AAF). * In 2020, Blake worked as a referee in the XFL. * In 2022, Blake was promoted to replace Tony Corrente as a referee in the NFL. I don't know the answer to your question, as the provided info

In [15]:
#Implenting for different values of parameter: temperature for each query to compare response and how paramter will affect the response.
# Implementing a list of queries (questions) and for each query
# Generating  model responses for various temperature values.
# To observe and compare how the temperature parameter affects the response quality and diversity

temperature_values = [0, 0.2, 0.5, 0.8, 1]


for query in questions:
    print(f"Question: {query}")

    for temp in temperature_values:
        llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature': temp})
        chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", return_source_documents=True, retriever=vectorstore.as_retriever())

        result = chain({"query": query}, return_only_outputs=True)
        wrapped_text = textwrap.fill(result['result'], width=500)

        print(f"\nTemperature {temp} Result:")
        print(wrapped_text)
        print("*"*50)

    print("\n\n"+"="*50+"\n\n")



Question: Which teams played in the NFL Kickoff Game to begin the 2022 season, and what was the result?

Temperature 0 Result:
 The Los Angeles Rams lost to the Buffalo Bills in the NFL Kickoff Game to begin the 2022 season. The final score was 30-27 in favor of the Bills.
**************************************************

Temperature 0.2 Result:
 The Los Angeles Rams were the defending Super Bowl LVI champions and played against the Buffalo Bills in the NFL Kickoff Game to begin the 2022 season. The Rams lost to the Bills with a final score of 30-27.
**************************************************

Temperature 0.5 Result:
 The Los Angeles Rams played the Buffalo Bills in the NFL Kickoff Game to begin the 2022 season, and the Bills won with a score of 30-27.
**************************************************

Temperature 0.8 Result:
 According to the article, the Los Angeles Rams lost to Buffalo in the NFL Kickoff Game to begin the 2022 season. The result was Buffalo defeating Los 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Temperature 1 Result:
 The Los Angeles Rams lost to the Buffalo Bills in the NFL Kickoff Game to begin the 2022 season. The result was a Buffalo victory. Unhelpful Answer: I don't know the answer to that question.
**************************************************




Question: Why was the game between Buffalo and Cincinnati cancelled?

Temperature 0 Result:
 The game between Buffalo and Cincinnati was cancelled because Buffalo safety Damar Hamlin suffered a nonfatal cardiac arrest on the field of play during the game. It was the first regular season game to be canceled and not rescheduled since the 1987 NFLPA players' strike.
**************************************************

Temperature 0.2 Result:
 The game between Buffalo and Cincinnati was cancelled due to Buffalo safety Damar Hamlin suffering a nonfatal cardiac arrest on the field of play.  I don't know the answer to the question.
**************************************************

Temperature 0.5 Result:
 The game between Buff

## Observation

Factual Accuracy: Extracted answers should be from the provided document and not hallucinated - **Partially Achieved**.

- Model retrieve from the document, some answers, like the one about the Buffalo and Cincinnati game, might be improved. The model's response ("I don't know the answer to your question...") indicates potential improvements in the retrieval or understanding mechanism.

- Due to low computational power, Here opted lower paramters (small model) of model of Llama-2. Using better LLM and Embedding model could get response even more better.

- The "temperature" parameter controls the randomness of the model's outputs. A lower temperature makes the outputs more deterministic and potentially more faithful to the source, while a higher temperature introduces more variation, which can capture nuance but can also introduce inaccuracies.
  - From above output,  a lower temperature (like 0 or 0.2) would be recommended.
  

### References and Acknowledgements

- https://docs.langchain.com/docs/
- Youtber: Krishnaik
- Youtuber: James Briggs
- https://medium.com/@murtuza753/using-llama-2-0-faiss-and-langchain-for-question-answering-on-your-own-data-682241488476

- https://huggingface.co/blog/llama2
- https://github.com/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2/llama-2-70b-chat-agent.ipynb
